In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import pickle
%matplotlib inline

In [2]:
from collections import namedtuple
from collections import defaultdict

In [3]:
sealion_types = ["adult_males", 
    "subadult_males",
    "adult_females",
    "juveniles",
    "pups"]

In [4]:
num_superpixels = [10, 8, 5, 4, 2]

In [5]:
expected_size = [72, 64, 56, 48, 20]

In [6]:
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (0, 255, 255)]

In [7]:
class Superpixel:
    def __init__(self):
        self.area = 0
        self.x_min = None
        self.x_max = None
        self.y_min = None
        self.y_max = None
        self.x_centroid = 0
        self.y_centroid = 0
        self.Lab_mean = 0
        
        
    def update(self, x, y, Lab):        
        self.area += 1
        if self.area == 1:
            self.x_min, self.x_max = x, x
            self.y_min, self.y_max = y, y
        else:
            self.x_min = min(self.x_min, x)
            self.y_min = min(self.y_min, y)
            self.x_max = max(self.x_max, x)
            self.y_max = max(self.y_max, y)
        
        self.x_centroid += x
        self.y_centroid += y
        self.Lab_mean += Lab
        
    @property
    def xc(self):
        if self.area != 0:            
            return self.x_centroid/self.area
        else:
            return None
    
    @property
    def yc(self):
        if self.area != 0:
            return self.y_centroid/self.area
        else:
            return None
        
    @property
    def Lab(self):
        if self.area != 0:            
            return self.Lab_mean/self.area
        else:
            return None
        
        
    def __str__(self):
        return ' - '.join(["({xmin}, {ymin}, {xmax}, {ymax})",
                           "({xc}, {yc})",
                           "{area}",
                           "{Lab}"])\
                    .format(xmin=self.x_min,
                            ymin=self.y_min,
                            xmax=self.x_max,
                            ymax=self.y_max,
                            xc=self.xc,
                            yc=self.yc,
                            area=self.area,
                            Lab=self.Lab)
        
    def __repr__(self):
        return self.__str__()

In [8]:
def opencvLAB(Lab):
    Lab32 = Lab.astype(np.float32)
    Lab32[..., 0] *= 100 / 255
    Lab32[..., 1] -= 128
    Lab32[..., 2] -= 128
    return Lab32

def deltaE(Lab1, Lab2):
    L1, a1, b1 = opencvLAB(Lab1)
    L2, a2, b2 = opencvLAB(Lab2)
    dL = L1 - L2
    da = a1 - a2
    db = b1 - b2
    return np.sqrt(dL * dL + da * da + db * db)

In [9]:
test_bgr = np.array([[[90, 78, 26]]], np.uint8)
test_lab_cv = cv2.cvtColor(test_bgr, cv2.COLOR_BGR2Lab)
test_lab = opencvLAB(test_lab_cv)

In [10]:
def stretch_hsv(im_bgr):
    im_hsv = cv2.cvtColor(im_bgr, cv2.COLOR_BGR2HSV)
    im_hsv_stretched = im_hsv.copy()
    for c in [1, 2]: # only for saturation and value
        min_channel = np.min(im_hsv[:,:,c])
        max_channel = np.max(im_hsv[:,:,c])
        a = 255.0/(max_channel - min_channel)
        b = -a * min_channel
        im_hsv_stretched[:,:,c] = (a * im_hsv[:,:,c] + b).astype(np.uint8)
    im_bgr_stretched = cv2.cvtColor(im_hsv_stretched, cv2.COLOR_HSV2BGR)
    return im_bgr_stretched

In [11]:
def analyze_superpixels(labels, im_Lab):
    labels_set = np.unique(labels)
    n_labels = labels_set.shape[0]
    
    h, w = labels.shape
    
    superpixels = defaultdict(Superpixel)
    neighbors = defaultdict(set)
    weights = defaultdict(int)
    dxs = [1, 0, -1, 0]
    dys = [0, 1, 0, -1]
    for y in range(h):
        for x in range(w):
            label = labels[y, x]
            superpixels[label].update(x, y, im_Lab[y, x])
            for dx, dy in zip(dxs, dys):
                if 0 <= x + dx < w and 0 <= y + dy < h:
                    n = labels[y + dy, x + dx]
                    if n != label:
                        neighbors[label].add(n)
                        weights[label] += 1
    return superpixels, neighbors, weights

In [36]:
def fit_ellipse(mask):
    y, x = np.nonzero(mask)
    coords = np.zeros((x.shape[0], 2), dtype=np.int32)
    coords[:, 0] = x
    coords[:, 1] = y
    #return cv2.fitEllipse(coords)
    return cv2.minAreaRect(coords)

In [61]:
def get_sealion_mask(dot, n_closest, labels, superpixels, neighbors):
    """ Find orientation of the sealion.
    
    Starting from the dot, we search for the n_closest closest superpixels
    and use them to get an approximate orientation.
    """
    x, y, = dot
    root_id = labels[y,x]
    sealion_ids = set([root_id])
    while len(sealion_ids) < n_closest:
        # Search the closest (in term of color) node from one of the nodes attributed to the sealion.
        closest_id = -1
        closest_distance = 1e9
        for node_id in sealion_ids:
            for neighbor_id in neighbors[node_id]:
                if neighbor_id not in sealion_ids:
                    distance_color = deltaE(superpixels[node_id].Lab, superpixels[neighbor_id].Lab)
                    if distance_color < closest_distance:
                        closest_distance = distance_color
                        closest_id = neighbor_id
        sealion_ids.add(closest_id)
    
    # Let's find the orientation of the cluster.
    mask = np.in1d(labels.ravel(),np.asarray(list(sealion_ids))).reshape(labels.shape)
    mask_u8 = (mask * 255).astype(np.uint8)
    mask_blur_u8 = cv2.blur(mask_u8, (15, 15))
    mask = (mask_blur_u8 > 27)
    return mask

In [62]:
def get_sealions_directions(root_dir, train_id):
    patch_size = 128
    
    im = cv2.imread(os.path.join(root_dir, "Train/{}.jpg".format(train_id)))
    h, w, c = im.shape
    
    ellipses = []
    with open(os.path.join(root_dir, "TrainDotsB/{}.pkl".format(train_id)), "rb") as pfile:
        dots = pickle.load(pfile)
    for i, ds in enumerate(dots):
        ellipses.append([])
        for (x, y) in ds:
            # Extract path around the dot
            x_start = max(0, x -  patch_size//2)
            y_start = max(0, y -  patch_size//2)
            x_end = x_start + patch_size
            y_end = y_start + patch_size
            if x_end >= w:
                dx = x_end - w + 1
                x_start -= dx
                x_end = x_start + patch_size
            if y_end >= h:
                dy = y_end - h + 1
                y_start -= dy
                y_end = y_start + patch_size
            patch = im[y_start:y_end, x_start:x_end,...]
            patch_normed = stretch_hsv(patch)
            
            # Find superpixels
            slico = cv2.ximgproc.createSuperpixelSLIC(patch_normed, cv2.ximgproc.SLICO, 10, 10.0)
            slico.iterate(20)
            labels = slico.getLabels()
            
            # Get the graph of superpixels in the patch
            im_lab = cv2.cvtColor(patch_normed, cv2.COLOR_BGR2Lab)
            im_lab = opencvLAB(im_lab)
            superpixels, neighbors, weights = analyze_superpixels(labels, im_lab)
            
            # Find the best ellipse
            mask = get_sealion_mask((x - x_start, y - y_start), num_superpixels[i], labels, superpixels, neighbors)
            e = fit_ellipse(mask)
            
            # Correct ellipse coordinates
            we, he = e[1]
            size = max(we, he)
            ratio = expected_size[i]/size
            e_corrected = ((x, y), (we * ratio, he * ratio), e[2])
            ellipses[i].append(e_corrected)
    return ellipses

In [63]:
def draw(im, ellipses):
    for i, es in enumerate(ellipses):
        for e in es:
            cv2.ellipse(im, e, colors[i], thickness=1)
            cv2.circle(im, e[0], 2, colors[i], thickness=-1)
    return im

In [64]:
root_dir = "/home/lowik/sealion/data/sealion/"

In [65]:
train_id = 8
ellipses = get_sealions_directions(root_dir, train_id)
im = cv2.imread("../data/sealion/Train/{}.jpg".format(train_id))
im_draw = draw(im.copy(), ellipses)
cv2.imwrite("../data/test.png", im_draw)

True

Process all train images

In [19]:
ellipses_dir = os.path.join(root_dir, "TrainEllipses")
os.makedirs(ellipses_dir, exist_ok=True)
for filename in os.listdir(os.path.join(root_dir, "Train")):
    if not filename.endswith(".jpg"):
        continue
    train_id, _ = os.path.splitext(filename)
    train_id = int(train_id)
    ellipses = get_sealions_directions(root_dir, train_id)
    with open(os.path.join(ellipses_dir, "{}.pkl".format(train_id)), "wb") as pfile:
        pickle.dump(ellipses, pfile, pickle.HIGHEST_PROTOCOL)

1019
987
579
529
561
548
494
462
537
372
466
476
513
487
341
411
374
503
458
479
717
826
524
489
450
468
432
485
344
423
432
334
375
296
369
423
339
269
1006
1074
1133
1084
1019
706
894
797
917
789
830
836
845
896
886
822
436
570
495
567
492
530
453
555
555
468
505
490
478
489
453
507
400
590
506
440
469
454
485
551
585
481
471
555
441
549
489
485
597
544
550
513
390
514
552
524
505
572
571
525
546
508
460
529
464
497
383
477
534
581
486
515
469
507
494
556
439
576
379
343
405
414
401
449
365
401
286
366
447
412
423
426
445
365
380
454
427
410
458
415
464
435
365
438
367
452
456
388
403
395
431
405
358
385
356
416
390
342
474
373
363
429
428
417
408
368
379
383
431
335
450
419
449
456
486
411
410
373
358
425
321
446
346
419
374
458
356
430
350
430
420
343
394


KeyboardInterrupt: 